In [ ]:
import pandas as pd

In [ ]:
file_loc = input("Enter the file path: ")

checklist = pd.read_csv(file_loc, sep="\t", on_bad_lines="warn")

In [ ]:
checklist

In [ ]:
# list all columns in dataframe

checklist.columns

In [ ]:
# drop columns datasetId, scientificNameAuthorship, genericName, specificEpithet, namePulishedIn, taxonRemarks

checklist = checklist.drop(
    [
        "datasetID",
        "scientificNameAuthorship",
        "genericName",
        "specificEpithet",
        "namePublishedIn",
        "taxonRemarks",
    ],
    axis=1,
)

In [ ]:
checklist.to_parquet("")

In [ ]:
checklist = pd.read_parquet("")

In [ ]:
checklist = checklist.drop(
    [
        "scientificName",
        "infraspecificEpithet",
        "nameAccordingTo",
        "nomenclaturalStatus",
    ],
    axis=1,
)

In [ ]:
# list unique values of taxonomicStatus

checklist["taxonomicStatus"].unique()

In [ ]:
# use taxonomicStatus and acceptedNameUsageID to sort

accepted = checklist[checklist["taxonomicStatus"] == "accepted"]

In [ ]:
accepted.to_parquet("")

In [ ]:
unaccepted = checklist[checklist["taxonomicStatus"] != "accepted"]

In [ ]:
unaccepted.to_parquet("")

In [ ]:
accepted = pd.read_parquet("")

In [ ]:
accepted

In [ ]:
accepted["acceptedNameUsageID"].unique()

In [ ]:
# find row where acceptedNameUsageID = 4335624

accepted[accepted["acceptedNameUsageID"] == 4335624]

In [ ]:
init = accepted[accepted["acceptedNameUsageID"].isnull()]

init

In [ ]:
# drop columns parentNameUsageID, acceptedNameUsageID, originalNameUsageID from init

init = init.drop(
    [
        "parentNameUsageID",
        "acceptedNameUsageID",
        "originalNameUsageID",
    ],
    axis=1,
)

init

In [ ]:
init["acceptedName"] = init["canonicalName"]

init

In [ ]:
cleanup = accepted[accepted["acceptedNameUsageID"].notnull()]

cleanup

In [ ]:
# create variable that contains the rows in acceptedwhere taxonID matches acceptedNameUsageID in cleanup

test = accepted[accepted["taxonID"].isin(cleanup["acceptedNameUsageID"])]

test

In [ ]:
# sort test based on ascending value of taxonID

test = test.sort_values(by="taxonID")

test

In [ ]:
# sort cleanup based on acceptedNameUsageID

cleanup = cleanup.sort_values(by="acceptedNameUsageID")

cleanup

In [ ]:
# convert acceptedNameUsageID to int in cleanup

cleanup["acceptedNameUsageID"] = cleanup["acceptedNameUsageID"].astype(int)


cleanup

In [ ]:
test["acceptedNameUsageID"] = test["taxonID"]


test

In [ ]:
# merge test and cleanup

merge = pd.merge(test, cleanup, on=["acceptedNameUsageID"], how="outer")


merge

In [ ]:
merge["acceptedName"] = merge["canonicalName_x"]


merge

In [ ]:
# drop all columns in merge that have x in the name

merge = merge.drop(
    [
        "taxonID_x",
        "parentNameUsageID_x",
        "originalNameUsageID_x",
        "canonicalName_x",
        "acceptedNameUsageID",
        "parentNameUsageID_y",
        "originalNameUsageID_y",
        "taxonRank_x",
        "taxonomicStatus_x",
        "kingdom_x",
        "phylum_x",
        "class_x",
        "order_x",
        "family_x",
        "genus_x",
    ],
    axis=1,
)


merge

In [ ]:
# rename all columns in merge that have y in the name

merge = merge.rename(
    columns={
        "taxonID_y": "taxonID",
        "parentNameUsageID_y": "parentNameUsageID",
        "originalNameUsageID_y": "originalNameUsageID",
        "canonicalName_y": "canonicalName",
        "taxonRank_y": "taxonRank",
        "taxonomicStatus_y": "taxonomicStatus",
        "kingdom_y": "kingdom",
        "phylum_y": "phylum",
        "class_y": "class",
        "order_y": "order",
        "family_y": "family",
        "genus_y": "genus",
    }
)


merge

In [ ]:
# concat init and merge

final = pd.concat([init, merge], axis=0)

In [ ]:
final.to_parquet("")

In [ ]:
df = pd.read_parquet("")

In [ ]:
df

In [ ]:
unaccepted = pd.read_parquet("")

In [ ]:
unaccepted

In [ ]:
# drop columns parentNameUsageID, originalNameUsageID,

unaccepted = unaccepted.drop(
    [
        "parentNameUsageID",
        "originalNameUsageID",
    ],
    axis=1,
)

unaccepted

In [ ]:
no_id = unaccepted[unaccepted["acceptedNameUsageID"].isnull()]

In [ ]:
unaccepted = unaccepted[unaccepted["acceptedNameUsageID"].notnull()]

In [ ]:
# convert acceptedNameUsageID to int

unaccepted["acceptedNameUsageID"] = unaccepted["acceptedNameUsageID"].astype(int)

unaccepted

In [ ]:
df["acceptedNameUsageID"] = df["taxonID"]

df

In [ ]:
merge = pd.merge(df, unaccepted, on=["acceptedNameUsageID"], how="outer")


merge

In [ ]:
merge.columns

In [ ]:
test = merge[merge["acceptedName"] == "Anaxyrus americanus"]

In [ ]:
test

In [ ]:
# remove all columns that have x in the name

merge = merge.drop(
    [
        "taxonID_x",
        "canonicalName_x",
        "taxonRank_x",
        "taxonomicStatus_x",
        "kingdom_x",
        "phylum_x",
        "class_x",
        "order_x",
        "family_x",
        "genus_x",
    ],
    axis=1,
)


merge

In [ ]:
# rename columns with y in the name

merge = merge.rename(
    columns={
        "taxonID_y": "taxonID",
        "taxonRank_y": "taxonRank",
        "taxonomicStatus_y": "taxonomicStatus",
        "canonicalName_y": "canonicalName",
        "kingdom_y": "kingdom",
        "phylum_y": "phylum",
        "class_y": "class",
        "order_y": "order",
        "family_y": "family",
        "genus_y": "genus",
    },
)


merge

In [ ]:
# create a new df with all the rows where acceptedName and canonicalName is null

is_null = merge[merge["acceptedName"].isnull() & merge["canonicalName"].isnull()]


is_null

In [ ]:
# remove all rows where acceptedName and canonicalName is null

merge = merge[~merge.isin(is_null).any(axis=1)]


merge

In [ ]:
merge["taxonRank"].unique()

In [ ]:
variety = merge[merge["taxonRank"] == "variety"]

variety

In [ ]:
subspecies = merge[merge["taxonRank"] == "subspecies"]

subspecies

In [ ]:
form = merge[merge["taxonRank"] == "form"]

form

In [ ]:
nan = merge[merge["taxonRank"].isnull()]

nan

In [ ]:
unranked = merge[merge["taxonRank"] == "unranked"]

unranked

In [ ]:
# remove all rows where taxonRank is kingdom, phylum, class, order, family, genus

merge = merge[
    ~merge["taxonRank"].isin(["kingdom", "phylum", "class", "order", "family", "genus"])
]


merge

In [ ]:
# remove rows where taxonRank is null and acceptedName is null or taxonRank is null and canonicalName is null

merge = merge[
    ~(
        merge["taxonRank"].isnull()
        & (merge["acceptedName"].isnull() | merge["canonicalName"].isnull())
    )
]


merge

In [ ]:
# remove rows where acceptedName is not 2 words

merge = merge[merge["acceptedName"].str.count(" ") == 1]

merge

In [ ]:
# remove duplicates from canonicalName column in merge

merge = merge.drop_duplicates(subset="canonicalName")


merge

In [ ]:
final = pd.concat([df, merge], axis=0)


final

In [ ]:
# these are different species with same scientific name
final_duplicates = final[final["canonicalName"].duplicated(keep=False)]

final_duplicates

In [ ]:
# remove columns taxonID and acceptedNameUsageID from final_duplicates

final_duplicates = final_duplicates.drop(["taxonID", "acceptedNameUsageID"], axis=1)

In [ ]:
final_duplicates.to_parquet("")

In [ ]:
# find rows where canonicalName is Thysananthus anguiformis

final_duplicates[final_duplicates["canonicalName"] == "Micrococcus antarcticus"]

In [ ]:
# remove rows from final_duplicates taxonomicStatus equals accepted

final_duplicates = final_duplicates[final_duplicates["taxonomicStatus"] != "accepted"]

final_duplicates

In [ ]:
final = final[final["acceptedName"].str.count(" ") == 1]

final

In [ ]:
# remove final_duplicates from final

final = final.drop(final_duplicates.index)

final

In [ ]:
# remove columns taxonID and acceptedNameUsageID from final

final = final.drop(["taxonID", "acceptedNameUsageID"], axis=1)

In [ ]:
final.to_parquet("")